In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from django.contrib.auth.models import User
from cancan.ability import Ability
from cancan.access_rules import AccessRules
from core.models import Project, Membership

Let's create some test data:

In [3]:
# delete some old models
User.objects.all().delete()
Project.objects.all().delete()

alice = User.objects.create(username="Alice", email="alice@acme.com")
bob = User.objects.create(username="Bob", email="bob@acme.com")

project1 = Project.objects.create(name="Alice's Project", created_by=alice)
project2 = Project.objects.create(name="Bob's project", created_by=bob)

project2.members.add(alice)

Now we have `project1` created by Alice with no other members, and `project2` created by Bob with Alice as a member.

Now let's declare user abilities function:

In [4]:
def declare_abilities(user, rules):
    # creators can view and edit projects
    rules.allow("view", Project, created_by=user)
    rules.allow("edit", Project, created_by=user)
    # members can view projects
    rules.allow("view", Project, members=user)

Now let's create the `ability` object for Alice. This is similar to what `can-can` middleware is doing under the hood.

In [5]:
access_rules = AccessRules(alice)
declare_abilities(alice, access_rules)
alice_ability = Ability(access_rules)

In [6]:
[alice_ability.can("view", project1), alice_ability.can("view", project2)]

[True, True]

In [7]:
[alice_ability.can("edit", project1), alice_ability.can("edit", project2)]

[True, False]

Now let's create the ability object for Bob

In [8]:
access_rules = AccessRules(bob)
declare_abilities(bob, access_rules)
bob_ability = Ability(access_rules)

In [9]:
[bob_ability.can("view", project1), bob_ability.can("view", project2)]

[False, True]

In [10]:
[bob_ability.can("edit", project1), bob_ability.can("edit", project2)]

[False, True]

Let's see the queryset for `view` action for both users:

In [11]:
print(alice_ability.queryset_for("view", Project).query)

SELECT "core_project"."id", "core_project"."name", "core_project"."description", "core_project"."created_by_id" FROM "core_project" LEFT OUTER JOIN "core_membership" ON ("core_project"."id" = "core_membership"."project_id") WHERE ("core_membership"."user_id" = 14 OR "core_project"."created_by_id" = 14)


In [12]:
print(bob_ability.queryset_for("view", Project).query)

SELECT "core_project"."id", "core_project"."name", "core_project"."description", "core_project"."created_by_id" FROM "core_project" LEFT OUTER JOIN "core_membership" ON ("core_project"."id" = "core_membership"."project_id") WHERE ("core_membership"."user_id" = 15 OR "core_project"."created_by_id" = 15)
